## Day 24

### Part 1

#### Test

In [1]:
test = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

In [2]:
nodes, gates = test.split("\n\n")
gates = gates.replace("AND", "&")
gates = gates.replace("XOR", "!=")
gates = gates.replace("OR", "|")

nodes = nodes.split("\n") 

nodes = [i.split(":") for i in nodes]
nodes = {i[0]: int(i[1].strip()) for i in nodes}

for k,v in nodes.items():
    gates = gates.replace(k, str(v))
    
gates = gates.split("\n") 

gates = [i.split("->") for i in gates]
gates = {i[1].strip(): i[0].strip().lower() for i in gates}

In [3]:
def update_nodes(nodes, gates):
    new_gates = {}
    for k,v in gates.items():
    
        try:
            nodes[k] = int(eval(v))
        except:
            new_gates[k] = v
    return nodes, new_gates

def update_gates(nodes, gates):
    new_gates = {}

    for k, v in gates.items():
        for kk, vv in nodes.items():
            v = v.replace(kk, str(vv))

        new_gates[k] = v

    return new_gates

In [4]:
while gates:
    
    nodes, gates = update_nodes(nodes, gates)
    gates = update_gates(nodes, gates)

In [5]:
result = ""
for k, v in sorted(nodes.items()):
    if k[0] == "z":
        result = str(v) + result

result = int(result, 2)

In [6]:
assert result == 2024

## Task

In [7]:
with open("../../../advent_of_code_input/2024/day_24/input.txt", "r") as f:
    data = f.read()

nodes, gates = data[:-1].split("\n\n")
gates = gates.replace("AND", "&")
gates = gates.replace("XOR", "!=")
gates = gates.replace("OR", "|")

nodes = nodes.split("\n") 

nodes = [i.split(":") for i in nodes]
nodes = {i[0]: int(i[1].strip()) for i in nodes}

for k,v in nodes.items():
    gates = gates.replace(k, str(v))
    
gates = gates.split("\n") 

gates = [i.split("->") for i in gates]
gates = {i[1].strip(): i[0].strip().lower() for i in gates}

In [8]:
while gates:
    
    nodes, gates = update_nodes(nodes, gates)
    gates = update_gates(nodes, gates)

In [9]:
result = ""
for k, v in sorted(nodes.items()):
    if k[0] == "z":
        result = str(v) + result

result = int(result, 2)

In [10]:
result

55920211035878

## Part 2

impossible to solve for me. i [copy & pasted this solution](https://www.reddit.com/r/adventofcode/comments/1hl698z/comment/m3l1rbe/), in the hope i will understand it some day.

## Task

In [11]:
with open("../../../advent_of_code_input/2024/day_24/input.txt", "r") as f:
    data = f.read()

nodes, gates = data[:-1].split("\n\n")    
gates = gates.split("\n") 

In [12]:
def find_gate(x_wire: str, y_wire: str, gate_type: str, configurations: list[str]):
    sub_str_a = f'{x_wire} {gate_type} {y_wire} -> '
    sub_str_b = f'{y_wire} {gate_type} {x_wire} -> '

    for config in configurations:
        if (sub_str_a in config) or (sub_str_b in config):
            return config.split(' -> ')[-1]


def swap_output_wires(wire_a: str, wire_b: str, configurations: list[str]):
    new_configurations = []

    for config in configurations:
        input_wires, output_wire = config.split(' -> ')

        if output_wire == wire_a:
            new_configurations.append(' -> '.join([input_wires] + [wire_b]))
        
        elif output_wire == wire_b:
            new_configurations.append(' -> '.join([input_wires] + [wire_a]))
        
        else:
            new_configurations.append(' -> '.join([input_wires] + [output_wire]))

    return new_configurations


def check_parallel_adders(configurations):
    current_carry_wire = None
    swaps = []
    bit = 0

    while True:
        x_wire = f'x{bit:02d}'
        y_wire = f'y{bit:02d}'
        z_wire = f'z{bit:02d}'

        if bit==0:
            current_carry_wire = find_gate(x_wire, y_wire, 'AND', configurations)
        else:
            ab_xor_gate = find_gate(x_wire, y_wire, 'XOR', configurations)
            ab_and_gate = find_gate(x_wire, y_wire, 'AND', configurations)

            cin_ab_xor_gate = find_gate(ab_xor_gate, current_carry_wire, 'XOR', configurations)
            if cin_ab_xor_gate is None:
                swaps.append(ab_xor_gate)
                swaps.append(ab_and_gate)
                configurations = swap_output_wires(ab_xor_gate, ab_and_gate, configurations)
                bit = 0
                continue
            
            if cin_ab_xor_gate != z_wire:
                swaps.append(cin_ab_xor_gate)
                swaps.append(z_wire)
                configurations = swap_output_wires(cin_ab_xor_gate, z_wire, configurations)
                bit = 0
                continue

            cin_ab_and_gate = find_gate(ab_xor_gate, current_carry_wire, 'AND', configurations)
            
            carry_wire = find_gate(ab_and_gate, cin_ab_and_gate, 'OR', configurations)
            current_carry_wire = carry_wire
        
        bit += 1
        if bit >= 45:
            break
    
    return swaps

In [13]:
swaps = check_parallel_adders(gates)
result = ','.join(sorted(swaps))

In [14]:
result

'btb,cmv,mwp,rdg,rmj,z17,z23,z30'